## 1st Trial

using covid19net from the tutorial to test the workflow
lossCriterion = CrossEntropyLoss
Optimizer = SGD
learning rate = 0.001
momentum = 0.9
epoch = 10
1st epoch got loss ~900
Finish with loss ~100
Accuracy is 97% on test set, which can happen, but with loss this high, its not good

Same covid19net
changes
-learning rate = 0.0025
-epoch = 20

1st epoch : loss = 984 ~ 1000
Finish with : loss =  